<a href="https://colab.research.google.com/github/mahopman/IEBM-Net/blob/main/intervention_classifier/generate_intervention_classifier_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
local_path = '/content/drive/MyDrive/MS_DataScience/DS595/IEBM-Net_Data'
evidence_integration_path = f'{local_path}/evidence_integration'
pretraining_dataset_path = f'{local_path}/pretraining_dataset'

In [7]:
import json

pmcid2intervention = json.load(open(f'{evidence_integration_path}/pmcid2intervention.json'))
pmcid2content = json.load(open(f'{evidence_integration_path}/evidence_inference/pmc_contents.json'))
sec2label = json.load(open(f'{evidence_integration_path}/evidence_inference/sec2label.json'))
secname2sec = json.load(open(f'{evidence_integration_path}/evidence_inference/secname2sec.json'))

dataset = {}
ctx_id = 0

for pmcid in pmcid2intervention:
    passage = ''
    if pmcid in pmcid2content:
        content = pmcid2content[pmcid]
        for secname, text in content:
            if secname[:len('ABSTRACT')] != 'ABSTRACT': continue
            if sec2label[secname2sec[secname]] == '1':
                passage += text

        dataset[ctx_id] = {}
        dataset[ctx_id]['passage'] = passage
        dataset[ctx_id]['intervention'] = pmcid2intervention[pmcid]
        ctx_id += 1

In [8]:
import glob

pmid2intervention = json.load(open(f'{pretraining_dataset_path}/pmid2intervention.json'))

files = glob.glob(f'{pretraining_dataset_path}/evidence/contexts_*.json')

for file in files:
    data = json.load(open(file))
    chunk_id = file.split('/')[-1].split('.')[0].split('_')[-1]
    print(f'Processing chunk #{chunk_id}')

    for entry in data:
        pmid = entry['pmid']
        if pmid in pmid2intervention:
            dataset[ctx_id] = {}
            dataset[ctx_id]['passage'] = entry['ctx']
            dataset[ctx_id]['intervention'] = pmid2intervention[pmid]
            ctx_id += 1

Processing chunk #0657
Processing chunk #0658
Processing chunk #0659
Processing chunk #0660
Processing chunk #0661
Processing chunk #0715
Processing chunk #0663
Processing chunk #0664
Processing chunk #0665
Processing chunk #0667
Processing chunk #0666
Processing chunk #0669
Processing chunk #0668
Processing chunk #0670
Processing chunk #0671
Processing chunk #0672
Processing chunk #0673
Processing chunk #0674
Processing chunk #0676
Processing chunk #0675
Processing chunk #0677
Processing chunk #0678
Processing chunk #0679
Processing chunk #0681
Processing chunk #0680
Processing chunk #0682
Processing chunk #0683
Processing chunk #0684
Processing chunk #0685
Processing chunk #0686
Processing chunk #0687
Processing chunk #0688
Processing chunk #0689
Processing chunk #0690
Processing chunk #0691
Processing chunk #0692
Processing chunk #0693
Processing chunk #0694
Processing chunk #0695
Processing chunk #0696
Processing chunk #0697
Processing chunk #0698
Processing chunk #0699
Processing 

In [14]:
import numpy as np
n = ctx_id
train_size = int(n * 0.8)
test_size = n - train_size

# Get indices for each section
train_indices = np.random.choice(n, train_size, replace=False)
test_indices = list(set(range(n)) - set(train_indices))

# Split the dataset
train = {k: v for k, v in dataset.items() if k in train_indices}
test = {k: v for k, v in dataset.items() if k in test_indices}

In [12]:
!mkdir -p {local_path}/intervention_classifier

In [15]:
with open(f'{local_path}/intervention_classifier/train.json', 'w') as f:
    json.dump(train, f)

with open(f'{local_path}/intervention_classifier/test.json', 'w') as f:
    json.dump(test, f)